# Naive Bayes Modeling

### Import libraries and data

In [10]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import string

C:\Users\zasz\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\zasz\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\zasz\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [11]:
# load data
train = pd.read_csv("train_clean.csv")
val = pd.read_csv("val_clean.csv")

## Preprocess data

In [12]:
# preprocess data

# remove punctuation and lowercase everything
def preprocess(text):
    for char in string.punctuation:
        text = text.replace(char, "")
    text = text.lower()
    return text

train.text = train.text.map(lambda x: preprocess(x))
val.text = val.text.map(lambda x: preprocess(x))

# split x and y
X_train, y_train = train.text, train.real_news
X_val, y_val = val.text, val.real_news

# tf-idv
vz = TfidfVectorizer(stop_words="english")
X_train = vz.fit_transform(X_train)
X_val = vz.transform(X_val)

## Grid Search

In [19]:
param_grid = {"max_depth":[5, 10, 15, 20], "learning_rate": np.linspace(0.3, 1, 3)}

gs = GridSearchCV(XGBClassifier(), param_grid)
gs.fit(X_train, y_train)

C:\Users\zasz\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     objective='binary:logistic',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                   

In [20]:
gs.best_params_

{'learning_rate': 0.6499999999999999, 'max_depth': 5}

In [21]:
model = gs.best_estimator_

## Evaluation

In [22]:
# fit and predict using model
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_val_hat = model.predict(X_val)

# classification reports
print("training performance")
print(classification_report(y_train, y_train_hat))
print("validation performance")
print(classification_report(y_val, y_val_hat))

training performance
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13757
           1       1.00      1.00      1.00     12796

    accuracy                           1.00     26553
   macro avg       1.00      1.00      1.00     26553
weighted avg       1.00      1.00      1.00     26553

validation performance
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4545
           1       0.99      0.99      0.99      4307

    accuracy                           0.99      8852
   macro avg       0.99      0.99      0.99      8852
weighted avg       0.99      0.99      0.99      8852

